### Upload input data to MinIO

In [1]:
import os
import boto3

s3 = boto3.resource('s3',
                    #endpoint_url='http://127.0.0.1:61403',
                    #endpoint_url='http://127.0.0.1:30080',
                    endpoint_url='https://minio-api.digitalhub-test.smartcommunitylab.it/',
                    aws_access_key_id='minio',
                    aws_secret_access_key='digitalhub-test',
                    aws_session_token=None,
                    config=boto3.session.Config(signature_version='s3v4'))
s3.buckets.all()
for bucket in s3.buckets.all():
    print(bucket.name)

datalake
ipzs


In [110]:
!ls  "/home/jovyan/ai_product_card_templates/document_classification_model/fasttext/ipzs"

1861  1876  1891  1906	1921  1936  1951  1966	1981  1996  2011
1862  1877  1892  1907	1922  1937  1952  1967	1982  1997  2012
1863  1878  1893  1908	1923  1938  1953  1968	1983  1998  2013
1864  1879  1894  1909	1924  1939  1954  1969	1984  1999  2014
1865  1880  1895  1910	1925  1940  1955  1970	1985  2000  2015
1866  1881  1896  1911	1926  1941  1956  1971	1986  2001  2016
1867  1882  1897  1912	1927  1942  1957  1972	1987  2002  2017
1868  1883  1898  1913	1928  1943  1958  1973	1988  2003  2018
1869  1884  1899  1914	1929  1944  1959  1974	1989  2004  2019
1870  1885  1900  1915	1930  1945  1960  1975	1990  2005  2020
1871  1886  1901  1916	1931  1946  1961  1976	1991  2006  2021
1872  1887  1902  1917	1932  1947  1962  1977	1992  2007  atti_SG_materie.csv
1873  1888  1903  1918	1933  1948  1963  1978	1993  2008
1874  1889  1904  1919	1934  1949  1964  1979	1994  2009
1875  1890  1905  1920	1935  1950  1965  1980	1995  2010


In [59]:
ipzs_bucket = s3.Bucket("ipzs")
input_folder = "/home/jovyan/ai_product_card_templates/document_classification_model/fasttext/ipzs"
years = ["2021", '1988', '1950', '1990']

for year in years:
    for root, subfolders, files in os.walk(input_folder + "/" + year):
        for item in files:
            if item.endswith(".json") or item.endswith(".csv"):
                fileNamePath = str(os.path.join(root,item))
                ipzs_bucket.upload_file(fileNamePath, fileNamePath.replace(input_folder + "/", ""))
ipzs_bucket.upload_file(input_folder + "/atti_SG_materie.csv" , "atti_SG_materie.csv")

In [3]:
for root, subfolders, files in os.walk(input_folder):
    for item in files:
        if item.startswith(".DS"):
            fileNamePath = str(os.path.join(root,item))
            print(fileNamePath)
            os.remove(fileNamePath)

### Configure MLRun

In [4]:
import mlrun

In [13]:
#set path of MLRun API running on Minikube
#mlrun.set_environment("http://127.0.0.1:30070")
mlrun.set_environment("http://localhost:8080")
#mlrun.set_environment("http://localhost:8060")
#set path of MLRun API running ok Kubernetes
#mlrun.set_environment("https://mlrun-api.digitalhub-test.smartcommunitylab.it", username="digitalhub-dev")

('default', '/home/jovyan/data/{{project}}')

In [14]:
mlrun.get_secret_or_env("MLRUN_DBPATH")

'http://localhost:8080'

### Create a project

In [15]:
#project = mlrun.new_project("document-classification", context="./", overwrite=True, init_git=False, user_project=False)
project = mlrun.get_or_create_project("text-classification-fasttext3", context="./", init_git=False, user_project=False)

> 2024-02-07 09:24:40,537 [warning] skipping setup, setup() handler was not found in project_setup.py
> 2024-02-07 09:24:40,538 [info] Project loaded successfully: {'project_name': 'text-classification-fasttext3'}


### Register the pre-processing function and run it

In [60]:
preproc_fn = project.set_function(
    name="pre-processing",
    func="01-preprocessing_handlers.py",
    handler="parse_ipzs",
    kind="job",
    image="mlrun/mlrun", #includes sklearn, pandas, numpy
    #requirements=[] #list or path to a requirements.txt
)

In [61]:
project.save()

In [62]:
import pandas as pd
df = pd.read_csv(
        f"s3://ipzs/atti_SG_materie.csv",
        delimiter=",",
        quotechar='"',
        storage_options={"client_kwargs": {"endpoint_url": "https://minio-api.digitalhub-test.smartcommunitylab.it/"}, 'key':'minio', 'secret':'digitalhub-test'}
    )
df

,REDAZ,DATAGU,TMAT,CODMAT
0,21G00103,23-JUN-21,MARE,A4130
1,21G00097,23-JUN-21,CONVENZIONI E TRATTATI INTERNAZIONALI,A1810
2,21G00108,23-JUN-21,GOVERNO,A3370
3,21G00096,22-JUN-21,CONVENZIONI E TRATTATI INTERNAZIONALI,A1810
4,21G00107,22-JUN-21,ECONOMIA NAZIONALE E PROGRAMMAZIONE ECONOMICA,A2530
...,...,...,...,...
12955,087G0754,05-JAN-88,PROFESSIONI INTELLETTUALI,A5250
12956,087G0755,05-JAN-88,REGISTRO (IMPOSTA DI),A5670
12957,087G0753,04-JAN-88,ALIMENTI E BEVANDE (IGIENE E COMMERCIO),A0290
12958,087G0742,02-JAN-88,"FIERE, MOSTRE ED ESPOSIZIONI",A3100


In [63]:
preproc_run = project.run_function(
    "pre-processing",
    #local=False,
    params={"bucket_name": "ipzs", "idPrefix": "ipzs-", "limit": 10, "max_documents": 250},
    outputs=["preprocessed_data"]
)

> 2024-02-07 10:59:54,967 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 10:59:54,969 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 10:59:54,975 [info] Storing function: {'name': 'pre-processing-parse-ipzs', 'uid': 'a6f33f4e7db84d46b1d756a44d3a096f', 'db': 'http://localhost:8080'}
> 2024-02-07 10:59:55,002 [debug] starting local run: 01-preprocessing_handlers.py # parse_ipzs
> 2024-02-07 10:59:55,016 [debug] The packagers manager collected the packager: BoolPackager(packable_type=bool, artifact_types=['object', 'result'], priority=5)
> 2024-02-07 10:59:55,017 [debug] The packagers manager collected the packager: BytearrayPackager(packable_type=bytearray, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 10:59:55,017 [debug] The packagers manager collected the packager: BytesPackager(packable_type=bytes, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 10:59:55,018 [de

INFO:root:s3_endpoint_url: https://minio-api.digitalhub-test.smartcommunitylab.it/
INFO:root:s3_endpoint_url:  provaaa
INFO:root:here1s3://ipzs/atti_SG_materie.csv
ERROR:root:20210623_21G00103 not in index
ERROR:root:20210623_21G00097 not in index
ERROR:root:20210623_21G00108 not in index
ERROR:root:20210622_21G00096 not in index
ERROR:root:20210622_21G00107 not in index
ERROR:root:20210621_21G00104 not in index
ERROR:root:20210618_21G00095 not in index
ERROR:root:20210617_21G00092 not in index
ERROR:root:20210616_21G00091 not in index
ERROR:root:20210615_21G00094 not in index
ERROR:root:20210614_21G00098 not in index
ERROR:root:20210611_21G00089 not in index
ERROR:root:20210609_21G00093 not in index
ERROR:root:20210608_21G00090 not in index
ERROR:root:20210604_21G00088 not in index
ERROR:root:20210531_21G00087 not in index
ERROR:root:20210531_21G00086 not in index
ERROR:root:20210528_21G00085 not in index
ERROR:root:20210526_21G00082 not in index
ERROR:root:20210525_21G00083 not in in

In [65]:
preproc_run.outputs["preprocessed_data"]

'store://artifacts/text-classification-fasttext3/pre-processing-parse-ipzs_preprocessed_data:a6f33f4e7db84d46b1d756a44d3a096f'

### Register the parsing function and run it

**NOTE**: building/auto-building images does not work on ARM because the resulting images are for ARM but AMD images are required. Manually building images and loading them on DockerHub is the current workaround.

In [66]:
#image has been built with:
# docker build -t classification-parsing:latest -<<EOF
# FROM mlrun/mlrun:1.2.1
# RUN pip install tqdm==4.61.1
# RUN pip install requests==2.25.1
# RUN pip install stanza==1.4.2
# EOF
parsing_fn = project.set_function(
    name="parsing",
    func="02-parsing_handlers.py",
    handler="parse",
    kind="job",
    image="ertomaselli/classification-parsing:latest"
)

#for VM with autobuild
# parsing_fn = project.set_function(
#     name="parsing",
#     func="02-parsing_handlers.py",
#     handler="parse",
#     kind="job",
#     image="mlrun/mlrun",
#     requirements=["tqdm==4.61.1", "requests==2.25.1", "stanza==1.4.2"] #list or path to a requirements.txt
# )

In [67]:
project.save()

In [68]:
!pip install stanza
parsing_run = project.run_function(
    "parsing",
    inputs={"input_file": preproc_run.outputs["preprocessed_data"]},
    params={"tint_url": None}
)


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
> 2024-02-07 11:00:49,084 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:00:49,087 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:00:49,099 [info] Storing function: {'name': 'parsing-parse', 'uid': 'b6c97fef1adb4d7bacb0c4226822c4e6', 'db': 'http://localhost:8080'}
> 2024-02-07 11:00:49,124 [debug] starting local run: 02-parsing_handlers.py # parse
> 2024-02-07 11:00:49,144 [debug] The packagers manager collected the packager: BoolPackager(packable_type=bool, artifact_types=['object', 'result'], priority=5)
> 2024-02-07 11:00:49,144 [debug] The packagers manager collected the packager: BytearrayPackager(packable_type=bytearray, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 11:00:49,145 [debug] The packagers manager collected the packager: BytesPackager(packable_type=bytes, a

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: it (Italian):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


> 2024-02-07 11:00:49,890 [error] execution error, Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/mlrun/runtimes/local.py", line 441, in exec_from_params
    val = mlrun.handler(
  File "/opt/conda/lib/python3.9/site-packages/mlrun/package/__init__.py", line 140, in wrapper
    func_outputs = func(*args, **kwargs)
  File "02-parsing_handlers.py", line 30, in parse
    os.mkdir(output_folder)
FileExistsError: [Errno 17] File exists: 'tint_files'

> 2024-02-07 11:00:49,907 [error] exec error - [Errno 17] File exists: 'tint_files'
> 2024-02-07 11:00:49,912 [debug] Run updates: {'name': 'parsing-parse', 'uid': 'b6c97fef1adb4d7bacb0c4226822c4e6', 'kind': '', 'last_state': 'error', 'updates': {'status.last_update': '2024-02-07T11:00:49.912055+00:00', 'status.state': 'error', 'status.error': "[Errno 17] File exists: 'tint_files'"}}
> 2024-02-07 11:00:49,920 [debug] No notifications to push for run: {'run_uid': 'b6c97fef1adb4d7bacb0c4226822c4e6'}


[Errno 17] File exists: 'tint_files'


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
text-classification-fasttext3,...6822c4e6,0,Feb 07 11:00:49,error,parsing-parse,kind=owner=jovyanhost=a7388fe5f22d,input_file,tint_url=None,,


> 2024-02-07 11:00:49,928 [info] Run execution finished: {'status': 'error', 'name': 'parsing-parse'}


RunError: [Errno 17] File exists: 'tint_files'

In [ ]:
parsing_run.outputs

### Register the function that extracts test sets and run it

In [69]:
extracting_fn = project.set_function(
    name="extracting_test",
    func="03-extracting_test_handlers.py",
    handler="extract_test_sets",
    kind="job",
    image="mlrun/mlrun"
)

Names with underscore '_' are about to be deprecated, use dashes '-' instead. Replacing underscores with dashes.


In [70]:
project.save()

In [71]:
extracting_run = project.run_function(
    "extracting_test",
    inputs={"input_file": preproc_run.outputs["preprocessed_data"], "tint_files": parsing_run.outputs["tint_files"]},
    params={"testRatio": 0.2, "devRatio": 0.2}
)

> 2024-02-07 11:00:52,795 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:00:52,796 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:00:52,802 [info] Storing function: {'name': 'extracting-test-extract-test-sets', 'uid': 'd85ecb1c6fc640b5a21af140b85d501e', 'db': 'http://localhost:8080'}
> 2024-02-07 11:00:52,822 [debug] starting local run: 03-extracting_test_handlers.py # extract_test_sets
> 2024-02-07 11:00:52,840 [debug] The packagers manager collected the packager: BoolPackager(packable_type=bool, artifact_types=['object', 'result'], priority=5)
> 2024-02-07 11:00:52,840 [debug] The packagers manager collected the packager: BytearrayPackager(packable_type=bytearray, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 11:00:52,841 [debug] The packagers manager collected the packager: BytesPackager(packable_type=bytes, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07

Names with underscore '_' are about to be deprecated, use dashes '-' instead. Replacing underscores with dashes.
INFO:root:Loading JSON file
INFO:root:Downloading and extracting tint_files.zip
INFO:root:Shuffling data
INFO:root:Extracting texts
INFO:root:Saving test list


> 2024-02-07 11:00:52,869 [debug] log artifact testlist at /home/jovyan/data/text-classification-fasttext3/extracting-test-extract-test-sets/0/testlist.txt, size: 0, db: Y


INFO:root:Saving dev list


> 2024-02-07 11:00:52,884 [debug] log artifact devlist at /home/jovyan/data/text-classification-fasttext3/extracting-test-extract-test-sets/0/devlist.txt, size: 0, db: Y


INFO:root:Train size: 0
INFO:root:Test size: 0
INFO:root:Dev size: 0


> 2024-02-07 11:00:52,905 [debug] Run updates: {'name': 'extracting-test-extract-test-sets', 'uid': 'd85ecb1c6fc640b5a21af140b85d501e', 'kind': '', 'last_state': '', 'updates': {'status.last_update': '2024-02-07T11:00:52.905057+00:00', 'status.state': 'completed'}}
> 2024-02-07 11:00:52,914 [debug] No notifications to push for run: {'run_uid': 'd85ecb1c6fc640b5a21af140b85d501e'}


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
text-classification-fasttext3,...b85d501e,0,Feb 07 11:00:52,completed,extracting-test-extract-test-sets,kind=owner=jovyanhost=a7388fe5f22d,input_filetint_files,testRatio=0.2devRatio=0.2,,testlistdevlist


> 2024-02-07 11:00:52,923 [info] Run execution finished: {'status': 'completed', 'name': 'extracting-test-extract-test-sets'}


In [72]:
extracting_run.outputs

{'testlist': 'store://artifacts/text-classification-fasttext3/extracting-test-extract-test-sets_testlist:d85ecb1c6fc640b5a21af140b85d501e',
 'devlist': 'store://artifacts/text-classification-fasttext3/extracting-test-extract-test-sets_devlist:d85ecb1c6fc640b5a21af140b85d501e'}

### Register the function for saving data and run it

In [73]:
saving_fn = project.set_function(
    name="saving_data",
    func="04-saving_data_handlers.py",
    handler="save_data",
    kind="job",
    image="ertomaselli/classification-parsing:latest"
)

Names with underscore '_' are about to be deprecated, use dashes '-' instead. Replacing underscores with dashes.


In [74]:
project.save()

In [75]:
saving_run = project.run_function(
    "saving_data",
    inputs={"input_file": preproc_run.outputs["preprocessed_data"],
            "test_list_file": extracting_run.outputs["testlist"],
            "dev_list_file": extracting_run.outputs["devlist"],
            "tint_files": parsing_run.outputs["tint_files"]}
)

> 2024-02-07 11:00:57,335 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:00:57,337 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:00:57,343 [info] Storing function: {'name': 'saving-data-save-data', 'uid': '4d43071fe0ef416a8af836a3258b5073', 'db': 'http://localhost:8080'}
> 2024-02-07 11:00:57,364 [debug] starting local run: 04-saving_data_handlers.py # save_data
> 2024-02-07 11:00:57,390 [debug] The packagers manager collected the packager: BoolPackager(packable_type=bool, artifact_types=['object', 'result'], priority=5)
> 2024-02-07 11:00:57,390 [debug] The packagers manager collected the packager: BytearrayPackager(packable_type=bytearray, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 11:00:57,391 [debug] The packagers manager collected the packager: BytesPackager(packable_type=bytes, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 11:00:57,391 [debug] Th

Names with underscore '_' are about to be deprecated, use dashes '-' instead. Replacing underscores with dashes.
INFO:root:Loading JSON file
INFO:root:Downloading and extracting tint_files.zip
INFO:root:Extracting texts
0it [00:00, ?it/s]
INFO:root:Saving file


> 2024-02-07 11:00:57,419 [debug] log artifact complete at /home/jovyan/data/text-classification-fasttext3/saving-data-save-data/0/complete.json, size: 2, db: Y
> 2024-02-07 11:00:57,436 [debug] Run updates: {'name': 'saving-data-save-data', 'uid': '4d43071fe0ef416a8af836a3258b5073', 'kind': '', 'last_state': '', 'updates': {'status.last_update': '2024-02-07T11:00:57.436388+00:00', 'status.state': 'completed'}}
> 2024-02-07 11:00:57,444 [debug] No notifications to push for run: {'run_uid': '4d43071fe0ef416a8af836a3258b5073'}


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
text-classification-fasttext3,...258b5073,0,Feb 07 11:00:57,completed,saving-data-save-data,kind=owner=jovyanhost=a7388fe5f22d,input_filetest_list_filedev_list_filetint_files,,,complete


> 2024-02-07 11:00:57,452 [info] Run execution finished: {'status': 'completed', 'name': 'saving-data-save-data'}


In [76]:
saving_run.outputs

{'complete': 'store://artifacts/text-classification-fasttext3/saving-data-save-data_complete:4d43071fe0ef416a8af836a3258b5073'}

### Register the filtering function and save it

In [77]:
filtering_fn = project.set_function(
    name="filtering",
    func="05-filtering_handlers.py",
    handler="filter",
    kind="job",
    image="ertomaselli/classification-parsing:latest"
)

In [78]:
project.save()

In [79]:
filtering_run = project.run_function(
    "filtering",
    inputs={"complete_json_file": saving_run.outputs["complete"]},
    params={"minFreq": 3}
)

> 2024-02-07 11:01:02,104 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:01:02,105 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:01:02,112 [info] Storing function: {'name': 'filtering-filter', 'uid': '86d691bf5a7d4bdd842d02a5a2be1ba6', 'db': 'http://localhost:8080'}
> 2024-02-07 11:01:02,135 [debug] starting local run: 05-filtering_handlers.py # filter
> 2024-02-07 11:01:02,149 [debug] The packagers manager collected the packager: BoolPackager(packable_type=bool, artifact_types=['object', 'result'], priority=5)
> 2024-02-07 11:01:02,150 [debug] The packagers manager collected the packager: BytearrayPackager(packable_type=bytearray, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 11:01:02,150 [debug] The packagers manager collected the packager: BytesPackager(packable_type=bytes, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 11:01:02,151 [debug] The packager

[Errno 17] File exists: 'filtering_files'


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
text-classification-fasttext3,...a2be1ba6,0,Feb 07 11:01:02,error,filtering-filter,kind=owner=jovyanhost=a7388fe5f22d,complete_json_file,minFreq=3,,


> 2024-02-07 11:01:02,201 [info] Run execution finished: {'status': 'error', 'name': 'filtering-filter'}


RunError: [Errno 17] File exists: 'filtering_files'

In [80]:
filtering_run.outputs

{'filtering_files': 'store://artifacts/text-classification-fasttext3/filtering-filter_filtering_files:0bb502f30fec425f88c673925c0348a3'}

### Register the training function and run it

In [81]:
#image has been built with:
# docker build -t classification-training:latest -<<EOF
# FROM mlrun/mlrun:1.2.1
# RUN apt-get update
# RUN apt-get install build-essential -y
# RUN pip install fasttext
# EOF
training_fn = project.set_function(
    name="training",
    func="training_handlers.py",
    handler="train",
    kind="job",
    image="ertomaselli/classification-training:latest"
)

In [82]:
project.save()

In [97]:
!pip install fasttext
training_run = project.run_function(
    "training",
    inputs={"training_files": filtering_run.outputs["filtering_files"]}
)


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
> 2024-02-07 11:25:03,324 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:25:03,342 [info] Storing function: {'name': 'training-train', 'uid': '6eaf7f5cc4424a70ac53421d93341514', 'db': 'http://localhost:8080'}
> 2024-02-07 11:25:03,376 [debug] starting local run: training_handlers.py # train
> 2024-02-07 11:25:03,399 [debug] The packagers manager collected the packager: BoolPackager(packable_type=bool, artifact_types=['object', 'result'], priority=5)
> 2024-02-07 11:25:03,399 [debug] The packagers manager collected the packager: BytearrayPackager(packable_type=bytearray, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 11:25:03,400 [debug] The packagers manager collected the packager: BytesPackager(packable_type=bytes, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 11:25:03,400 [debug] The packag

INFO:root:Downloading and extracting filtering_files.zip
INFO:root:Creating goodTokens model
Read 0M words
Number of words:  66
Number of labels: 1
Progress: 100.0% words/sec/thread:  217862 lr:  0.000000 avg.loss:  0.000000 ETA:   0h 0m 0s
INFO:root:Testing goodTokens model
INFO:root:Predicting with goodTokens_unfiltered_model.bin model for the whole test data
INFO:root:Logging goodTokens artifacts


> 2024-02-07 11:25:03,469 [debug] log artifact goodTokens_unfiltered_model at /home/jovyan/data/text-classification-fasttext3/training-train/0/goodTokens_unfiltered_model/, size: 28160, db: Y


INFO:root:Creating goodTokens_by_document model
Read 0M words
Number of words:  1
Number of labels: 1
INFO:root:Testing goodTokens_by_document model
Progress: 100.0% words/sec/thread:   57756 lr:  0.000000 avg.loss:  0.000000 ETA:   0h 0m 0s
INFO:root:Predicting with goodTokens_by_document_filtered_model.bin model for the whole test data
INFO:root:Logging goodTokens_by_document artifacts


goodTokens_by_document_filtered.test.txt
           labels  texts
0  __label__A6040    NaN
1  __label__A6040    NaN
2  __label__A6040    NaN
3  __label__A6040    NaN




> 2024-02-07 11:25:03,528 [debug] log artifact goodTokens_by_document_filtered_model at /home/jovyan/data/text-classification-fasttext3/training-train/0/goodTokens_by_document_filtered_model/, size: 964, db: Y


...

allTokens_by_label_filtered.test.txt
           labels  texts
0  __label__A6040    NaN
1  __label__A6040    NaN
2  __label__A6040    NaN
3  __label__A6040    NaN




> 2024-02-07 11:25:04,076 [debug] log artifact allTokens_by_label_filtered_model at /home/jovyan/data/text-classification-fasttext3/training-train/0/allTokens_by_label_filtered_model/, size: 964, db: Y


INFO:root:Creating zipped folder to log as artifact


> 2024-02-07 11:25:04,097 [debug] log artifact results at /home/jovyan/data/text-classification-fasttext3/training-train/0/results.zip, size: 2758, db: Y
> 2024-02-07 11:25:04,129 [debug] Run updates: {'name': 'training-train', 'uid': '6eaf7f5cc4424a70ac53421d93341514', 'kind': '', 'last_state': '', 'updates': {'status.last_update': '2024-02-07T11:25:04.129810+00:00', 'status.state': 'completed'}}
> 2024-02-07 11:25:04,139 [debug] No notifications to push for run: {'run_uid': '6eaf7f5cc4424a70ac53421d93341514'}


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
text-classification-fasttext3,...93341514,0,Feb 07 11:25:03,completed,training-train,kind=owner=jovyanhost=a7388fe5f22d,training_files,,,goodTokens_unfiltered_modelgoodTokens_by_document_filtered_modelgoodTokens_by_label_filtered_modelallLemmas_unfiltered_modelallLemmas_by_document_filtered_modelallLemmas_by_label_filtered_modelallTokens_unfiltered_modelallTokens_by_document_filtered_modelallTokens_by_label_filtered_modelresults


> 2024-02-07 11:25:04,148 [info] Run execution finished: {'status': 'completed', 'name': 'training-train'}


In [98]:
training_run.outputs

{'goodTokens_unfiltered_model': 'store://artifacts/text-classification-fasttext3/training-train_goodTokens_unfiltered_model:6eaf7f5cc4424a70ac53421d93341514',
 'goodTokens_by_document_filtered_model': 'store://artifacts/text-classification-fasttext3/training-train_goodTokens_by_document_filtered_model:6eaf7f5cc4424a70ac53421d93341514',
 'goodTokens_by_label_filtered_model': 'store://artifacts/text-classification-fasttext3/training-train_goodTokens_by_label_filtered_model:6eaf7f5cc4424a70ac53421d93341514',
 'allLemmas_unfiltered_model': 'store://artifacts/text-classification-fasttext3/training-train_allLemmas_unfiltered_model:6eaf7f5cc4424a70ac53421d93341514',
 'allLemmas_by_document_filtered_model': 'store://artifacts/text-classification-fasttext3/training-train_allLemmas_by_document_filtered_model:6eaf7f5cc4424a70ac53421d93341514',
 'allLemmas_by_label_filtered_model': 'store://artifacts/text-classification-fasttext3/training-train_allLemmas_by_label_filtered_model:6eaf7f5cc4424a70ac5

### Register the evaluation function and run it

In [99]:
evaluation_fn = project.set_function(
    name="evaluation",
    func="06-evaluation_handlers.py",
    handler="evaluate",
    kind="job",
    image="mlrun/mlrun"
)

In [100]:
project.save()

In [101]:
evaluation_run = project.run_function(
    "evaluation",
    inputs={"pred_files": training_run.outputs["results"], "gold_files": filtering_run.outputs["filtering_files"]},
    params={"show_cm": True}
)

> 2024-02-07 11:25:10,185 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:25:10,187 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:25:10,193 [info] Storing function: {'name': 'evaluation-evaluate', 'uid': '175261f473594d6bb1ee3ce46ede1158', 'db': 'http://localhost:8080'}
> 2024-02-07 11:25:10,219 [debug] starting local run: 06-evaluation_handlers.py # evaluate
> 2024-02-07 11:25:10,245 [debug] The packagers manager collected the packager: BoolPackager(packable_type=bool, artifact_types=['object', 'result'], priority=5)
> 2024-02-07 11:25:10,246 [debug] The packagers manager collected the packager: BytearrayPackager(packable_type=bytearray, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 11:25:10,246 [debug] The packagers manager collected the packager: BytesPackager(packable_type=bytes, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 11:25:10,246 [debug] The pa

INFO:root:Downloading and extracting pred_files
INFO:root:Downloading and extracting gold_files
INFO:root:Pred file: results/goodTokens_unfiltered.results.txt
INFO:root:Test file: filtering_files/goodTokens_unfiltered.test.txt
INFO:root:Pred file: results/allLemmas_by_label_filtered.results.txt
INFO:root:Test file: filtering_files/allLemmas_by_label_filtered.test.txt
INFO:root:Pred file: results/allTokens_by_label_filtered.results.txt
INFO:root:Test file: filtering_files/allTokens_by_label_filtered.test.txt
INFO:root:Pred file: results/goodTokens_by_document_filtered.results.txt
INFO:root:Test file: filtering_files/goodTokens_by_document_filtered.test.txt
INFO:root:Pred file: results/goodTokens_by_label_filtered.results.txt
INFO:root:Test file: filtering_files/goodTokens_by_label_filtered.test.txt
INFO:root:Pred file: results/allLemmas_by_document_filtered.results.txt
INFO:root:Test file: filtering_files/allLemmas_by_document_filtered.test.txt
INFO:root:Pred file: results/allTokens_by_

> 2024-02-07 11:25:10,380 [debug] Run updates: {'name': 'evaluation-evaluate', 'uid': '175261f473594d6bb1ee3ce46ede1158', 'kind': '', 'last_state': '', 'updates': {'status.last_update': '2024-02-07T11:25:10.380004+00:00', 'status.state': 'completed'}}
> 2024-02-07 11:25:10,388 [debug] No notifications to push for run: {'run_uid': '175261f473594d6bb1ee3ce46ede1158'}


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
text-classification-fasttext3,...6ede1158,0,Feb 07 11:25:10,completed,evaluation-evaluate,kind=owner=jovyanhost=a7388fe5f22d,pred_filesgold_files,show_cm=True,goodTokens_unfiltered_macro=1.0goodTokens_unfiltered_micro=1.0goodTokens_unfiltered_weighted=1.0goodTokens_unfiltered_confusion_matrix={}allLemmas_by_label_filtered_macro=1.0allLemmas_by_label_filtered_micro=1.0allLemmas_by_label_filtered_weighted=1.0allLemmas_by_label_filtered_confusion_matrix={}allTokens_by_label_filtered_macro=1.0allTokens_by_label_filtered_micro=1.0allTokens_by_label_filtered_weighted=1.0allTokens_by_label_filtered_confusion_matrix={}goodTokens_by_document_filtered_macro=1.0goodTokens_by_document_filtered_micro=1.0goodTokens_by_document_filtered_weighted=1.0goodTokens_by_document_filtered_confusion_matrix={}goodTokens_by_label_filtered_macro=1.0goodTokens_by_label_filtered_micro=1.0goodTokens_by_label_filtered_weighted=1.0goodTokens_by_label_filtered_confusion_matrix={}allLemmas_by_document_filtered_macro=1.0allLemmas_by_document_filtered_micro=1.0allLemmas_by_document_filtered_weighted=1.0allLemmas_by_document_filtered_confusion_matrix={}allTokens_by_document_filtered_macro=1.0allTokens_by_document_filtered_micro=1.0allTokens_by_document_filtered_weighted=1.0allTokens_by_document_filtered_confusion_matrix={}allTokens_unfiltered_macro=1.0allTokens_unfiltered_micro=1.0allTokens_unfiltered_weighted=1.0allTokens_unfiltered_confusion_matrix={}allLemmas_unfiltered_macro=1.0allLemmas_unfiltered_micro=1.0allLemmas_unfiltered_weighted=1.0allLemmas_unfiltered_confusion_matrix={},


> 2024-02-07 11:25:10,395 [info] Run execution finished: {'status': 'completed', 'name': 'evaluation-evaluate'}


In [102]:
evaluation_run.outputs

{'goodTokens_unfiltered_macro': 1.0,
 'goodTokens_unfiltered_micro': 1.0,
 'goodTokens_unfiltered_weighted': 1.0,
 'goodTokens_unfiltered_confusion_matrix': {},
 'allLemmas_by_label_filtered_macro': 1.0,
 'allLemmas_by_label_filtered_micro': 1.0,
 'allLemmas_by_label_filtered_weighted': 1.0,
 'allLemmas_by_label_filtered_confusion_matrix': {},
 'allTokens_by_label_filtered_macro': 1.0,
 'allTokens_by_label_filtered_micro': 1.0,
 'allTokens_by_label_filtered_weighted': 1.0,
 'allTokens_by_label_filtered_confusion_matrix': {},
 'goodTokens_by_document_filtered_macro': 1.0,
 'goodTokens_by_document_filtered_micro': 1.0,
 'goodTokens_by_document_filtered_weighted': 1.0,
 'goodTokens_by_document_filtered_confusion_matrix': {},
 'goodTokens_by_label_filtered_macro': 1.0,
 'goodTokens_by_label_filtered_micro': 1.0,
 'goodTokens_by_label_filtered_weighted': 1.0,
 'goodTokens_by_label_filtered_confusion_matrix': {},
 'allLemmas_by_document_filtered_macro': 1.0,
 'allLemmas_by_document_filtered_

### Define and run a pipeline

In [103]:
project.set_workflow(
    "classification",
    workflow_path="classification_pipeline.py",
    engine="kfp",
    handler="classification_pipeline"
)

In [104]:
project.save()

In [106]:
run_id = project.run(
    name="classification",
    # arguments={
    #     "data_format": "parse_ipzs",
    #     "bucket_name": "ipzs", "idPrefix": "ipzs-", "limit": 10, "max_documents": 100,
    #     "tint_url": None,
    #     "testRatio": 0.2, "devRatio": 0.2
    # }, 
    watch=True
)

> 2024-02-07 11:26:36,505 [warning] running kubeflow pipeline locally, note some ops may not run locally!
> 2024-02-07 11:26:36,507 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:26:36,507 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:26:36,508 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:26:36,509 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:26:36,509 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:26:36,510 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:26:36,510 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:26:36,511 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:26:36,511 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2

> 2024-02-07 11:26:36,515 [debug] Auto mount disabled due to user selection (auto_mount_type=none)
> 2024-02-07 11:26:36,515 [info] Storing function: {'name': 'pre-processing-parse-ipzs', 'uid': 'd0ff4a086b3348b1a77391227b857954', 'db': None}
> 2024-02-07 11:26:36,539 [debug] starting local run: 01-preprocessing_handlers.py # parse_ipzs
> 2024-02-07 11:26:36,550 [debug] The packagers manager collected the packager: BoolPackager(packable_type=bool, artifact_types=['object', 'result'], priority=5)
> 2024-02-07 11:26:36,550 [debug] The packagers manager collected the packager: BytearrayPackager(packable_type=bytearray, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 11:26:36,551 [debug] The packagers manager collected the packager: BytesPackager(packable_type=bytes, artifact_types=['file', 'object', 'result'], priority=5)
> 2024-02-07 11:26:36,551 [debug] The packagers manager collected the packager: DictPackager(packable_type=dict, artifact_types=['file', 'object', 

INFO:root:s3_endpoint_url: https://minio-api.digitalhub-test.smartcommunitylab.it/
INFO:root:s3_endpoint_url:  provaaa
INFO:root:here1s3://ipzs/atti_SG_materie.csv
ERROR:root:20210623_21G00103 not in index
ERROR:root:20210623_21G00097 not in index
ERROR:root:20210623_21G00108 not in index
ERROR:root:20210622_21G00096 not in index
ERROR:root:20210622_21G00107 not in index
ERROR:root:20210621_21G00104 not in index
ERROR:root:20210618_21G00095 not in index
ERROR:root:20210617_21G00092 not in index
ERROR:root:20210616_21G00091 not in index
ERROR:root:20210615_21G00094 not in index
ERROR:root:20210614_21G00098 not in index
ERROR:root:20210611_21G00089 not in index
ERROR:root:20210609_21G00093 not in index
ERROR:root:20210608_21G00090 not in index
ERROR:root:20210604_21G00088 not in index
ERROR:root:20210531_21G00087 not in index
ERROR:root:20210531_21G00086 not in index
ERROR:root:20210528_21G00085 not in index
ERROR:root:20210526_21G00082 not in index
ERROR:root:20210525_21G00083 not in in

### Deploy the models

In [117]:
serving_fn = mlrun.new_function(
    "model-server", 
    kind="serving", 
    image="ertomaselli/classification-tqdm-stanza-fasttext:latest", 
    project="document-classification"
)

#model_path = training_run.outputs["allTokens_unfiltered_model"]
model_path = "./allTokens_unfiltered_model.bin" #test with single local model

# set the topology/router and add models
graph = serving_fn.set_topology("router")
serving_fn.add_model("allTokens_unfiltered_model", model_path=model_path, class_name="model_serving.ClassifierModel")

project.set_function(serving_fn)
project.save()

In [118]:
#test function locally
server = serving_fn.to_mock_server()

> 2024-02-07 12:34:14,460 [warning] run command, file or code were not specified


INFO:root:extra_data: {}


> 2024-02-07 12:34:14,473 [info] model allTokens_unfiltered_model was loaded
> 2024-02-07 12:34:14,473 [info] Loaded ['allTokens_unfiltered_model']


In [113]:
serving_fn.invoke("/v2/models/allTokens_unfiltered_model/infer", body={"inputs": [csv_path, text]})

> 2024-02-07 11:57:12,431 [info] invoking function: {'method': 'POST', 'path': 'http://192.168.49.1:32771/v2/models/allTokens_unfiltered_model/infer'}


OSError: error: cannot run function at url http://192.168.49.1:32771/v2/models/allTokens_unfiltered_model/infer, HTTPConnectionPool(host='192.168.49.1', port=32771): Max retries exceeded with url: /v2/models/allTokens_unfiltered_model/infer (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f28e51a0bb0>: Failed to establish a new connection: [Errno 111] Connection refused'))